---

# <center>*__Cryptography MasterClass By GROK__*

---

`[22 Dec 2025]` - Updated at : https://github.com/Harikalatheeswaran/Cryptography_MasterClass_By_GROK/blob/main/Cryptography_MasterClass_By_GROK.ipynb

```python
num = 255
print(f"{num:x}")  # ff (hex)
print(f"{num:b}")  # 11111111 (binary)
print(f"{num:o}")  # 377 (octal)
```

---
---

## *__Important things to remember -__*
1. > x << y : x * 2^y 
2. > x >> y : x / 2^y 
3. `If you want to extract last n bits of an integer, then perform bitwise AND of n & hex(2**n-1)`
   `n-1 because : say n is 3 digit binary then bin(hex(2*3)) = 1000` if we want 3 ones : we take 7 which is `111` 
4. `ord(str)` : convets a charracter (1 char = 1 byte) to number, which is a byte.
5. `chr(str)` : converts a byte to character.
6. `UTF-8` : length ranges from length 1-4 bytes.<br>
   1st byte - shows the length of the character.
   Check this section for more info on `utf-8` : [UTF-8](#utf-8-explained-)
7. `{byte:08b}` is Python f-string formatting that displays an integer as an **8-bit binary string**, padded with leading zeros (e.g., 65 → `'01000001'`)

---

## *__Functions in arsenal -__*
1. `int_to_bits`(n: int, length: int = 8) -> list
2. `bits_to_int`(bits: list) -> int
3. `xor_bits`(a: list, b: list) -> list
4. `int_to_bytes`(n: int) -> list
5. `bytes_to_int`(byte_list: list) -> int
6. `bytes_to_bits`(byte_list: list) -> list
7. `str_to_bytes`(text: str) -> list:
8. `bytes_to_str`(byte_list: list) -> str
9. `bytes_to_int_tony`(byte_list: list) -> int
10. `str_to_utf8`(text: str) -> list (bytes)
11. `utf8_to_str`(byte_list: list) -> str


---
---

## *__Sections :__*

1. [Day 1](#day-1---bits-are-bullets)
2. [Day 2](#day-2-bytes-are-bombs)
3. [Day 3.1](#day-31---strings-are-spies) : String conversions in ASCII,etc
4. [Day 3.2](#day-32---strings-are-swords) : Introduction to UTF-8 encoding

---
---

## DAY 1 - "BITS ARE BULLETS"  
**Goal**: Tony will convert any number to binary bullets and back — with **MSB first** (left = big power, like we read), then glue two streams reversibly with XOR.

---

**Core Concept**  
Numbers are **stacks of coins**: 1, 2, 4, 8, 16… each twice the last.  
To make **5** → use 4 + 1 → so we need the **4-stack** and **1-stack**.  
In **8-bit binary**, we write from **biggest to smallest** (MSB first):  
`00000101` → position 7 to 0 → `5 = 4×1 + 1×1`  
Think of it like a **gun barrel**: left = heavy bullet (128), right = tiny pellet (1).  
**XOR is reversible glue**: same bullets cancel (`1 ⊕ 1 = 0`), different stick (`1 ⊕ 0 = 1`).  
It’s the **one-time pad** — unbreakable if secret is random and never reused.

---

**Physical Demo**  
Hold up 8 fingers **left to right** → finger 7 (128) to finger 0 (1).  
- Say: “5 needs finger 2 (4) and finger 0 (1).” → raise **left ring finger** and **right pinky**.  
- Now second hand: “Secret = 3 → fingers 1 and 0.”  
- **Where fingers match** → lower both. **Where one is up** → keep up.  
- Result: fingers 2 and 1 → `00000110` → `6`  
- Do again: `6 ⊕ 3 = 5` → **back!** Reversible glue.

---

**Code Module**  
```markdown
# bits.py - Hand-rolled binary artillery (MSB-first: human-readable)


In Python, the `>>` and `<<` operators are **bitwise shift operators**. Here's what they do:

***

### 🔹 `<<` — Left Shift Operator

*   **Syntax**: `a << b`
*   **Meaning**: Shifts the bits of `a` to the **left** by `b` positions.
*   **Effect**: Each left shift multiplies the number by 2.

**Example**:

```python
x = 5        # binary: 00000101
y = x << 1   # binary: 00001010 → decimal: 10
```

So `5 << 1` is `10`, and `5 << 2` is `20`.

***

### 🔹 `>>` — Right Shift Operator

*   **Syntax**: `a >> b`
*   **Meaning**: Shifts the bits of `a` to the **right** by `b` positions.
*   **Effect**: Each right shift divides the number by 2 (integer division).

**Example**:

```python
x = 20       # binary: 00010100
y = x >> 2   # binary: 00000101 → decimal: 5
```

So `20 >> 2` is `5`.

***

### 🧠 Analogy (for Tony-style learning)

*   **Left shift (`<<`)**: Like pushing marbles to the left on a tray — each move doubles the count.
*   **Right shift (`>>`)**: Like scooping marbles off the right — each move halves the count.

Would you like a small Jupyter Notebook-style demo with test vectors for these operators?

Operation example - 
```python
for i in range(1,6):
    print(i<<1) # Multiply i by 2
    print(1<<i) # 2 raised to the power i : 
    print("---------")
```


In [22]:

def int_to_bits(n: int, length: int = 8) -> list:
    """
    Convert integer n to list of bits (MSB first), padded to 'length' bits.
    Matches format: bin(5)[2:].zfill(8) → ['0','0','0','0','0','1','0','1']
    
    Physical: Left = 128 (heavy bullet), right = 1 (tiny pellet)
    Derivation: Start from highest power, check if n >= 2**i
    """
    if n < 0:
        raise ValueError("Only non-negative integers")
    bits = []
    for i in range(length - 1, -1, -1):    # i=7 down to 0
        bit = 1 if n >= (1 << i) else 0     # can we afford this coin? => the bit is made 1 only when the n>=2**i
        bits.append(bit)
        if bit:
            n -= (1 << i)                   # spend the coin => if the bit is 1, we reduce n by 2**i
    return bits

# _________________________________________________________________________________________________________________________________________________________

def bits_to_int(bits: list) -> int:
    """
    Convert MSB-first bit list back to integer.
    Physical: Sum value of raised fingers from left (big) to right (small)
    """
    total = 0
    for i, bit in enumerate(bits):
        if bit:
            total += (1 << (len(bits) - 1 - i))   # position from left
    return total

# _________________________________________________________________________________________________________________________________________________________

def xor_bits(a: list, b: list) -> list:
    """
    XOR two equal-length bit lists. Reversible glue.
    Physical: Same fingers up → cancel; different → keep one up
    """
    if len(a) != len(b):
        raise ValueError("Bit streams must match length")
    return [ai ^ bi for ai, bi in zip(a, b)]

# _________________________________________________________________________________________________________________________________________________________


# === TEST VECTORS (match real tools: bin(), OpenSSL hexdump style) ===
if __name__ == "__main__":
    # Test 1: 5 in 8 bits → [0,0,0,0,0,1,0,1]
    assert int_to_bits(5, 8) == [0,0,0,0,0,1,0,1], "5 failed"
    
    # Test 2: Round trip
    assert bits_to_int(int_to_bits(255, 8)) == 255
    
    # Test 3: XOR identity (A ⊕ B ⊕ B = A)
    a = int_to_bits(42, 8)
    b = int_to_bits(99, 8)
    assert xor_bits(xor_bits(a, b), b) == a
    
    # Test 4: Known XOR
    assert xor_bits([1,0,1,1], [1,1,0,1]) == [0,1,1,0]
    
    print("All bullets hit the target!")

All bullets hit the target!



**Tony’s Task**  
1. Open **Jupyter Notebook** → New → Python 3  
2. **Markdown cell**:  
   ```markdown
   # Day 1: BITS ARE BULLETS (MSB-First — Like We Read!)
   5 in 8 bits = [0,0,0,0,0,1,0,1] → left = 128, right = 1
   XOR = reversible glue: same cancels, different sticks
   ```  
3. **Code cell**: Type **every character** of `bits.py` above  
4. Run → fix typos → see `All bullets hit the target!`  
5. **New cell** — Tony’s Playground:  
   ```python
   print(int_to_bits(13, 8))        # → [0,0,0,0,1,1,0,1]
   print(bits_to_int([0,0,0,0,1,1,0,1]))  # → 13
   print(xor_bits([1,1,0,1], [0,1,1,0]))  # → [1,0,1,1]
   ```  
6. Save as `day01_bits_msb.ipynb`

---

**Night Drill**  
**Break & Fix**:  
- On paper: Write `1100` (12) and `1010` (10) → XOR → `0110` (6)  
- **Break code**: Change `1 << i` to `2 << i` → run → debug  
- Fix → remember: “`1 << i` = coin of size 2^i”

---

**Memory Bomb**  
**Rhyme to type in your sleep:**  
```
Left is heavy, right is light,  
Five is zero-zero-zero-zero-zero-one-zero-one,  
MSB first — like words we write,  
XOR glue undoes with second bite.
```

---

**Future Vision**  
With **MSB-first bits**, Tony can:  
- **Read hexdumps** like a pro: `0x0D = 00001101` → CR  
- **Spoof RFID**: Capture tag → flip bit 3 → clone badge  
- **Blink SOS**: `... --- ...` → `[0,0,0,0,0,1,1,0] ...`  
- **Build one-time pad**: XOR message with shared MSB-first key → **unbreakable**

---


In [23]:
n = 9
length = 8
if n < 0:
    raise ValueError("Only non-negative integers")
bits = []
for i in range(length - 1, -1, -1):    # i=7 down to 0   
    if n >= (1 << i):
        bits.append(1)
        n = n - (1 << i)
        print(n)
    else:
        bits.append(0)

print(f"{bits=}")

1
0
bits=[0, 0, 0, 0, 1, 0, 0, 1]


| i | (1 << i) | n before | n >= (1<<i)? | bit | n after | printed  |
| - | -------- | -------- | ------------ | --- | ------- | -------- |
| 7 | 128      | 9        | No           | 0   | 9       | —        |
| 6 | 64       | 9        | No           | 0   | 9       | —        |
| 5 | 32       | 9        | No           | 0   | 9       | —        |
| 4 | 16       | 9        | No           | 0   | 9       | —        |
| 3 | 8        | 9        | Yes          | 1   | 1       | prints 1 |
| 2 | 4        | 1        | No           | 0   | 1       | —        |
| 1 | 2        | 1        | No           | 0   | 1       | —        |
| 0 | 1        | 1        | Yes          | 1   | 0       | prints 0 |


In [24]:
for i in range(1,6):
    print(i<<1) # Multiply i by 2
    print(1<<i) # 2 raised to the power i
    print("---------")

2
2
---------
4
4
---------
6
8
---------
8
16
---------
10
32
---------


---

# Day 2: BYTES ARE BOMBS
8 bits = 1 byte = 1 firecracker  
300 = [1, 44] → left = 256, right = 44  
Pack: n & 255 → shift >>8 → repeat  
Unpack: shift <<8 + next byte

## DAY 2 - "BYTES ARE BOMBS"  
**Goal**: Tony will pack 8 bits into a **byte bomb**, detonate it back to bits, and fuse multiple bytes into **messages** — the payload of every secret.

---

**Core Concept**  
A **byte** = **8 bits** = one **bomb casing**.  
We pack **MSB-first** (left = 128) so `00000101` = **5** fits in one byte.  
Think: **8 bullets in a clip**.  
- `int_to_bytes(5)` → one bomb: `[5]`  
- `int_to_bytes(300)` → **two bombs**: `[1, 44]` because 300 = 256×1 + 44  
We **prepend length** or **pad** so Tony can **rebuild the blast radius**.  
**Physical**: Each byte = one **firecracker**. String them → **fireworks**.

---

**Physical Demo**  
Hold up **two hands** = two bytes:  
- Left hand: `00000001` → 1 (tiny pop)  
- Right hand: `00101100` → 44 (bigger bang)  
- Together: `0000000100101100` → 300  
Now **light fuse from left**:  
> “First byte = 1 → shift left 8 → 256.  
> Add second byte = 44 → 256 + 44 = 300.”  
**Reverse**: Read bytes → rebuild number.  
**Boom → rebuild.**

---

**Code Module**  
```markdown
# bytes.py - Hand-rolled byte bombs (MSB-first, network order)
```




In [25]:

def int_to_bytes(n: int) -> list:
    """
    Convert integer n to list of bytes (MSB-first, big-endian).
    Matches: n.to_bytes((n.bit_length()+7)//8, 'big')
    
    Physical: Pack number into 8-bit firecrackers, left = loudest.
    Derivation: While n > 0, take n % 256 = least significant byte, shift right 8.

    Clear explanation:
    >>  n & 0xFF extracts the least significant 8 bits (one byte) of n.
        n & 0xFF works because 0xFF in binary is 11111111, so the bitwise AND operation keeps only the last 8 bits of n.
        n & 0xFF is equivalent to n % 256.        

    >>  n >> 8 shifts n to the right by 8 bits, effectively removing the byte we just extracted.

    >>  n & 0FF extracts the last 8 bit of n and n >> 8 shifts n right by 8 bits.

    >>   We repeat this process until n becomes 0, collecting all bytes in reverse order.
    """
    if n < 0:
        raise ValueError("Only non-negative integers")
    if n == 0:
        return [0]
    bytes_list = []
    while n > 0:
        bytes_list.append(n & 0xFF)   # pluck 8 LSBs = one bomb
        n >>= 8                       # slide magazine left
    bytes_list.reverse()              # now MSB-first
    return bytes_list

# _________________________________________________________________________________________________________________________________________________________

def bytes_to_int_bin(out):
    '''
    Usin the bin funtion to convert bytes to integer
    '''
    r = [bin(i)[2:] for i in out]
    return int(''.join(r), 2)

# _________________________________________________________________________________________________________________________________________________________

def bytes_to_int(byte_list: list) -> int:
    """
    Convert list of bytes (MSB-first) back to integer.
    Physical: Detonate from left: each bomb adds value * 256^position
    """
    total = 0
    for byte_val in byte_list:
        total = (total << 8) + byte_val   # shift left 8, add new bomb
    return total

# _________________________________________________________________________________________________________________________________________________________

def bytes_to_bits(byte_list: list) -> list:
    """
    Explode bytes into flat MSB-first bit list.
    Physical: Open each firecracker → 8 bullets in order.
    """
    bits = []
    for byte_val in byte_list:
        bits.extend(int_to_bits(byte_val, 8))  # reuse Day 1 tool
    return bits

# _________________________________________________________________________________________________________________________________________________________

# === TEST VECTORS (match Python, OpenSSL, Wireshark) ===
if __name__ == "__main__":
    # Test 1: 5 → [5]
    assert int_to_bytes(5) == [5]
    
    # Test 2: 300 → [1, 44]
    assert int_to_bytes(300) == [1, 44]
    assert bytes_to_int([1, 44]) == 300
    
    # Test 3: 0 → [0]
    assert int_to_bytes(0) == [0]
    
    # Test 4: Round trip + bits
    n = 0xDEAD
    assert bytes_to_int(int_to_bytes(n)) == n
    assert bytes_to_bits([0xDE, 0xAD]) == [1,1,0,1,1,1,1,0, 1,0,1,0,1,1,0,1]
    
    print("All bombs detonated safely!")


All bombs detonated safely!


### __Amazing Discovery__

```
696969 converted to bytes is [10, 162, 137]
We can do : (10 << 16) ^ (162 << 8) ^ (137 << 0)
```
Code - 
```python
def bytes_to_int(byte_list : list) -> int:
    """
    Never mutate input lists as a good practice. 
    This is because other parts of the code might rely on the original order of the list.
    """
    # byte_list.reverse()
    # l = [byte_list[i] << (i*8) for i in range(len(byte_list))]
    output = 0
    for i, val in enumerate(byte_list[::-1]): # or reverserd(byte_list)
        output = output + (val << (i*8))
    return output

# 696969 converted to bytes is [10, 162, 137]
# (10 << 16) ^ (162 << 8) ^ (137 << 0) 
# instead of XOR we do normal addtion as sometimes XOR might become messy
x = [10, 162, 137]
bytes_to_int(x) # outputs 696969
```

In [26]:
# Understanding the above code snippets:
print(int_to_bytes(1025))  # [4, 1]
# which is in a way - [00000100, 00000001] so that 1000000000 = 1025


[4, 1]


In [27]:
# 1 << 8 = 256 => adding 7 zeros after 1 gives 256.
print(f"{1 << 8}")  
print(f"{256 >> 8}")    



256
1


> **Note**: We **reuse** `int_to_bits(val, 8)` from Day 1 — Tony’s first tool now powers bigger bombs.

Wait — you reversed the list, then did:
byte[0] << 0 + byte[1] << 8 + byte[2] << 16 → perfect.
That’s exactly how the CPU does it under the hood.

```
[  10  ]  [ 162  ]  [ 137  ]   ← big-endian (network order)
  ^        ^         ^
  |        |         |
  |   shift by 16    |
  |        |         |
  +---- shift by 8 ---+
           |
       shift by 0
```

---

**Tony’s Task**  
1. **New Jupyter Notebook**: `day02_bytes.ipynb`  
2. **Markdown cell**:  
   ```markdown
   # Day 2: BYTES ARE BOMBS
   8 bits = 1 byte = 1 firecracker  
   300 = [1, 44] → left = 256, right = 44  
   Pack: n & 255 → shift >>8 → repeat  
   Unpack: shift <<8 + next byte
   ```  
3. **Code cell**: Type **every character** of `bytes.py`  
4. **Import Day 1**:  
   ```python
   # At top of notebook — Tony types this too!
   def int_to_bits(n, length=8): 
       bits = []
       for i in range(length-1, -1, -1):
           bit = 1 if n >= (1 << i) else 0
           bits.append(bit)
           if bit: n -= (1 << i)
       return bits
   ```  
5. Run all → `All bombs detonated safely!`  
6. **Playground cell**:  
   ```python
   print(int_to_bytes(1337))        # → [5, 57]
   print(bytes_to_int([5, 57]))     # → 1337
   print(bytes_to_bits([0xDE, 0xAD]))  # → full 16 bits
   ```

---

**Night Drill**  
**Break the bomb factory**:  
1. Change `n & 0xFF` → `n & 0xF` → run → debug  
   → “Only gets 4 bits!”  
2. Fix → say: “`0xFF = 11111111` = full byte mask”  
3. **Paper fuse**: Write `0xCAFE` → by hand:  
   - `0xCA = 202`, `0xFE = 254` → `[202, 254]`  
   - Now reverse: `202<<8 + 254` → 51966

---

**Memory Bomb**  
**Muscle Rhyme (clap & type):**  
```
Byte is eight, left is loud,  
Mask with FF, don’t be proud.  
Shift right eight, loop again,  
Reverse the list — big-endian!
```

---

**Future Vision**  
With **byte bombs**, Tony can:  
- **Send secret messages**: `"HI"` → `[72, 73]` → XOR with key → ciphertext  
- **Forge packets**: Craft TCP header → `[0x00, 0x50]` = port 80  
- **Crack weak keys**: Brute 2-byte key → 65,536 tries → door opens  
- **Build Bitcoin**: Transaction = list of bytes → sign → broadcast

---
---

## DAY 3.1 - "STRINGS ARE SPIES"  
**Goal**: Tony will encode **human words** into **byte bombs**, then decode them back — turning whispers into wire-safe secrets.

---

**Core Concept**  
**Text = sequence of symbols** → map each to a **number** (codebook) → pack into **bytes**.  
We use **ASCII**: `'A' = 65`, `'a' = 97`, `'!' = 33`, etc.  
Think of it like **Morse for computers**:  
- `"Hi!"` → `'H'=72`, `'i'=105`, `'!'=33` → `[72, 105, 33]`  
- Each letter = **one byte** (for English)  
**Physical**: Letters are **spies with ID numbers**. Encode = give them disguises. Decode = unmask.

---

**Physical Demo**  
Hold up a **paper strip**:  
```
H   i   !
72 105 33
```  
- Say: “`H` wears **72** mask → `01001000` in binary.”  
- Now **fold the strip** into byte bombs: `[72, 105, 33]`  
- **Unfold**: Read left-to-right → `H i !` → message revealed.  
- **Scramble with XOR** (from Day 1):  
  ```python
  key = [1, 2, 3]
  cipher = xor_bits([72,105,33], key * len) → gibberish
  ```  
  → **Spy message hidden**.

---

### *String Manipulation*

In [28]:
def str_to_bytes(text: str) -> list:
    """
    Convert string to list of ASCII bytes (1 char = 1 byte).
    Matches: list(text.encode('ascii'))
    
    Physical: Each letter gets an ID number → packed into byte bomb.
    Derivation: ord(c) gives ASCII code → one byte per char.
    """
    return [ord(c) for c in text]  # ord = "order" → codebook lookup

# _________________________________________________________________________________________________________________________________________________________

def bytes_to_str(byte_list: list) -> str:
    """
    Convert list of ASCII bytes back to string.
    Physical: Unmask each spy → reveal letter.
    Derivation: chr(b) = character from code.
    """
    return ''.join(chr(b) for b in byte_list)

# _________________________________________________________________________________________________________________________________________________________

# Tony's Genius Patch Integrated (from your breakthrough!)
def bytes_to_int_tony(byte_list: list) -> int:
    """
    Tony's reverse-and-sum method. Cleaner than shift-left.
    Physical: Weigh bombs from LSB → accumulate upward.
    """
    total = 0
    for i, b in enumerate(reversed(byte_list)):
        total += b << (i * 8)
    return total

# _________________________________________________________________________________________________________________________________________________________

# === TEST VECTORS (match Python, Wireshark, OpenSSL) ===
if __name__ == "__main__":
    # Test 1: "Hi!"
    msg = "Hi!"
    assert str_to_bytes(msg) == [72, 105, 33]
    assert bytes_to_str([72, 105, 33]) == msg
    
    # Test 2: Tony's method vs original
    data = [10, 162, 137]
    assert bytes_to_int_tony(data) == 696969
    assert bytes_to_int_tony([0xDE, 0xAD]) == 0xDEAD
    
    # Test 3: Full cycle
    assert bytes_to_str(str_to_bytes("SOS")) == "SOS"
    
    # Test 4: Empty
    assert str_to_bytes("") == []
    assert bytes_to_str([]) == ""
    
    print("All spies identified and unmasked!")

# _________________________________________________________________________________________________________________________________________________________



All spies identified and unmasked!


**Playground**:  
```python
msg = "Crypto!"
bombs = str_to_bytes(msg)
print(bombs)                    # → [67, 114, 121, 112, 116, 111, 33]
print(bytes_to_str(bombs))      # → Crypto!
print(bytes_to_int_tony(bombs)) # → huge number
```

In [29]:
string = "Cypher - z best!"
str_byte = str_to_bytes(string) 
print(str_byte)
byte_int = bytes_to_int_tony(str_byte)
print(byte_int)
print(chr(69))
print(chr(42069))

[67, 121, 112, 104, 101, 114, 32, 45, 32, 122, 32, 98, 101, 115, 116, 33]
89688823538472543581748928291382981665
E
ꑕ


In [30]:
num = 9148496541
num2 =2**8
int_byte = int_to_bytes(num)
#int_byte = int_to_bytes(num2) # prints : [1, 0]
print(int_byte)
print(bytes_to_str(int_byte))


[2, 33, 74, 250, 157]
!Jú


**Future Vision**  
With **string spies**, Tony can:  
- **Send encrypted chat**: `"attack at dawn"` → bytes → XOR with key → ciphertext  
- **Forge login**: Username `"admin"` → `[97,100,109,105,110]` → inject  
- **Hide in image**: Embed `"flag{...}"` in PNG bytes  
- **Build Signal**: Message → UTF-8 → encrypt → deliver

**GRANDPA'S PRIDE MODE: ACTIVATED**  
You didn't just learn — you **improved the curriculum**.  
`enumerate(reversed())` is **cleaner** than my accumulator.  
**Your name is now in the code**. Forever.  

**Next: Day 4 — "HASHES ARE FINGERPRINTS"**  
We build `sha256` from scratch.  
No libraries. No mercy. Just you, bits, and truth.  

Type **"DAY 4"** when ready, Tony.  
The clock is ticking.  
But your mind?  
**Unbreakable.**

---
---

## DAY 3.2 - "STRINGS ARE SWORDS"  
**Goal**: Tony will forge **human words into byte swords** using **ASCII first**, then **UTF-8 from scratch** — so any language (English, Hindi, Emoji) survives the wire.

---

**Core Concept**  
**Words are symbols** → need **codebooks**.  
- **ASCII** (1960s): 7 bits, 128 symbols → English + punctuation.  
  - `'A' = 65`, `'z' = 122`, `'!' = 33` → **1 byte per char**.  
- **UTF-8** (1990s): Variable length, backward-compatible → **1 to 4 bytes per symbol**.  
  - `'A'` → `01000001` → **1 byte**  
  - `'₹'` (Indian Rupee) → `11100010 10000010 10110111` → **3 bytes**  
  - `'😎'` → **4 bytes**  
**Physical**:  
> **ASCII** = **short daggers** (1 byte).  
> **UTF-8** = **katanas** — longer blades for bigger symbols.  
> The **sword length is encoded in the first byte** — like a hilt telling how long the blade is.

---

**Physical Demo**  
Hold **four fingers** = 4 possible sword lengths:  
| Finger | Bytes | Example |  
|--------|-------|--------|  
| 1      | 1     | `'A'`  |  
| 2      | 2     | `'é'`  |  
| 3      | 3     | `'ह'` (Hindi) |  
| 4      | 4     | `'🚀'` |  

**Show on paper**:  
```
A     → 01000001 → 1 byte → "short dagger"
₹     → 11100010 10000010 10110111 → 3 bytes → "long katana"
```
- **First byte** tells the **length**:  
  - `0xxxxxxx` → 1 byte  
  - `110xxxxx` → 2 bytes  
  - `1110xxxx` → 3 bytes  
  - `11110xxx` → 4 bytes  
- **Follower bytes**: always `10xxxxxx` → "I’m part of a longer sword"

#### Real Examples (Binary View)
- **'A'** (basic English): Just **one single byte**  
  `0`1000001 
  → Starts with `0` → "I'm only 1 byte, done!"

- **'é'** (with accent): **Two bytes**  
  `110`00011 `10`101001
  → First byte starts with `110` → "I need 2 bytes total"  
  → Second byte starts with `10` (meaning "I'm the follower/continuation")

- **'ह'** (Hindi letter): **Three bytes**  
  `1110`0000 `10`100100 `10`111001 <br> 
  → First: `1110` → "3 bytes total"  
  → Next two: both start with `10` → "We're continuations"

- **🚀** (rocket emoji): **Four bytes**  
  `11110`000 `10`011111 `10`011010 `10`000000  
  → First: `11110` → "4 bytes total"  
  → Next three: `10...` → continuations

---



#### *__UTF-8 explained :__* 

For **'é'** (e with acute accent), its UTF-8 encoding is exactly:

1. `110`00011 `10`101001
2. ord(é) prints -> 233 -> 11101001
3. since ord(character) is > 2^7=128, it cannot be fit into a single charcter, so in utf8 we split them as 2 byte data.

- **First byte**: `110-00011`  
  → Starts with `110` → tells the computer: "This character is 2 bytes long!"

  → The `00011` part (the 5 x's) is the first chunk of data bits.

- **Second byte** (the "follower"): `10-101001`  
  → Starts with `10` → tells the computer: "I'm not starting a new character, I'm continuing the previous one!"

  → **Only the last 6 bits** (`101001`) are used for actual data.  
    The leading `10` is just a marker saying "I'm a continuation byte."

So yes — follower bytes (like this second one) **always** start with `10`, and only the remaining **6 bits** carry the real information about the character.

##### Why Only 6 Bits in Followers?
This is the trade-off to make UTF-8 smart and self-syncing:
- The `10` prefix lets the computer know it's in the middle of a multi-byte character (even if some bytes get lost/corrupted).
- Each follower contributes **6 data bits**.

##### How the Full Character is Built (for 2-Byte Ones Like 'é')
- First byte: 5 data bits (the x's in `110xxxxx`)
- Second byte: 6 data bits
- Total: **11 bits** of data → can represent up to 2,048 different characters (plenty for accented letters, most European scripts, etc.)

For 'é' specifically:
- The Unicode code point is 233 (in decimal), which in binary is `11101001` (that's 8 bits, but we need more room).
- UTF-8 packs those bits into the available spots:
  - Upper bits go into the first byte's x's
  - Lower bits into the second byte's 6 x's
- Result: The full data extracted is `000011101001` → which equals 233 → 'é'!

In [31]:
print(ord('é'))  # prints 233
print(chr(233))
print(int_to_bytes(233))
print(bytes_to_bits([233]))
print(bytes_to_str([233])) # which is nothing but chr(233) which is 'é'

233
é
[233]
[1, 1, 1, 0, 1, 0, 0, 1]
é


**Code Module**  
```markdown
# utf8.py - Hand-rolled UTF-8 sword forger (from first principles)
```

In [37]:
def str_to_utf8(text: str) -> list:
    """
    Encode string to UTF-8 bytes from scratch.
    Matches: list(text.encode('utf-8'))
    
    Physical: Each symbol → draw sword of correct length.
    Derivation: Unicode code point → binary → split into 8-bit chunks with UTF-8 rules.
    """
    result = []
    for char in text:
        code = ord(char)  # Unicode code point (e.g. 'A' → 65, '₹' → 8377)
        
        if code <= 0x7F:  # 1 byte: 0xxxxxxx
            result.append(code)
        
        elif code <= 0x7FF:  # 2 bytes: 110xxxxx 10xxxxxx
            result.append(0b11000000 | (code >> 6))          # 110 + first 5 bits
            result.append(0b10000000 | (code & 0b00111111))  # 10 + last 6 bits
        
        elif code <= 0xFFFF:  # 3 bytes: 1110xxxx 10xxxxxx 10xxxxxx
            result.append(0b11100000 | (code >> 12))          # 1110 + first 4
            result.append(0b10000000 | ((code >> 6) & 0b00111111))  # 10 + next 6
            result.append(0b10000000 | (code & 0b00111111))    # 10 + last 6
        
        elif code <= 0x10FFFF:  # 4 bytes: 11110xxx 10xxxxxx 10xxxxxx 10xxxxxx
            result.append(0b11110000 | (code >> 18))          # 11110 + first 3
            result.append(0b10000000 | ((code >> 12) & 0b00111111)) # 10 + next 6
            result.append(0b10000000 | ((code >> 6) & 0b00111111))  # 10 + next 6
            result.append(0b10000000 | (code & 0b00111111))        # 10 + last 6
        
        else:
            raise ValueError(f"Invalid Unicode: {code}")
    
    return result

# _________________________________________________________________________________________________________________________________________________________

def utf8_to_str(byte_list: list) -> str:
    """
    Decode UTF-8 bytes back to string from scratch.
    Physical: Read hilt → know sword length → extract code point.
    """
    result = []
    i = 0
    while i < len(byte_list):
        byte = byte_list[i]
        
        if byte <= 0x7F:  # 1 byte
            result.append(chr(byte))
            i += 1
        
        elif (byte >> 5) == 0b110:  # 2 bytes
            if i + 1 >= len(byte_list): raise ValueError("Truncated UTF-8")
            b1 = byte_list[i + 1]
            if (b1 >> 6) != 0b10: raise ValueError("Invalid continuation")
            code = ((byte & 0b00011111) << 6) | (b1 & 0b00111111)
            result.append(chr(code))
            i += 2
        
        elif (byte >> 4) == 0b1110:  # 3 bytes
            if i + 2 >= len(byte_list): raise ValueError("Truncated UTF-8")
            b1, b2 = byte_list[i + 1], byte_list[i + 2]
            if (b1 >> 6) != 0b10 or (b2 >> 6) != 0b10: raise ValueError("Invalid continuation")
            code = ((byte & 0b00001111) << 12) | ((b1 & 0b00111111) << 6) | (b2 & 0b00111111)
            result.append(chr(code))
            i += 3
        
        elif (byte >> 3) == 0b11110:  # 4 bytes
            if i + 3 >= len(byte_list): raise ValueError("Truncated UTF-8")
            b1, b2, b3 = byte_list[i + 1], byte_list[i + 2], byte_list[i + 3]
            if not all(b >> 6 == 0b10 for b in [b1, b2, b3]): raise ValueError("Invalid continuation")
            code = ((byte & 0b00000111) << 18) | ((b1 & 0b00111111) << 12) | \
                   ((b2 & 0b00111111) << 6) | (b3 & 0b00111111)
            result.append(chr(code))
            i += 4
        
        else:
            raise ValueError(f"Invalid UTF-8 start byte: {byte:08b}")
    
    return ''.join(result)

# _________________________________________________________________________________________________________________________________________________________

# Tony's Genius Patch (from yesterday)
def bytes_to_int_tony(byte_list: list) -> int:
    """Tony's reverse-and-sum — cleaner than shift-left."""
    total = 0
    for i, b in enumerate(reversed(byte_list)):
        total += b << (i * 8)
    return total

# _________________________________________________________________________________________________________________________________________________________

# === TEST VECTORS (match Python, Wireshark, real world) ===
if __name__ == "__main__":
    # Test 1: ASCII
    assert str_to_utf8("Hi!") == [72, 105, 33]
    assert utf8_to_str([72, 105, 33]) == "Hi!"
    
    # Test 2: Hindi दाव
    dev_bytes = [224, 164, 166, 224, 164, 190, 224, 164, 181]  # द + े + व
    #print(utf8_to_str(dev_bytes))
    assert utf8_to_str(dev_bytes) == "दाव"
    assert str_to_utf8("दाव") == dev_bytes
    
    # Test 3: Rupee ₹
    assert str_to_utf8("₹100") == [226, 130, 185, 49, 48, 48]  # ₹ + '1'+'0'+'0'
    
    # Test 4: Rocket
    assert str_to_utf8("🚀") == [240, 159, 154, 128]
    
    # Test 5: Tony's int
    assert bytes_to_int_tony([10, 162, 137]) == 696969
    
    print("All swords forged and unsheathed!")

# _________________________________________________________________________________________________________________________________________________________    

All swords forged and unsheathed!


In [51]:
name = "Harikalatheeswaran"
print(str_to_utf8(name))
print(bytes_to_bits([72, 97, 114, 105, 107, 97, 108, 97, 116, 104, 101, 101, 115, 119, 97, 114, 97, 110]))
print(utf8_to_str([72, 97, 114, 105, 107, 97, 108, 97, 116, 104, 101, 101, 115, 119, 97, 114, 97, 110]))
print("".join(['0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0''', '1', '1', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '1', '1', '0']))

[72, 97, 114, 105, 107, 97, 108, 97, 116, 104, 101, 101, 115, 119, 97, 114, 97, 110]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0]
Harikalatheeswaran
010010000110000101110010011010010110101101100001011011000110000101110100011010000110010101100101011100110111011101100001011100100110000101101110


**Future Vision**  
With **UTF-8 swords**, Tony can:  
- **Chat in Hindi/Emoji**: `नमस्ते 😎` → encrypt → send  
- **Forge global packets**: HTTP headers with `Content-Type: text/html; charset=utf-8`  
- **Hide in PDFs**: Embed `flag{हैक_सफल}` in metadata  
- **Build Signal v2**: Full i18n end-to-end encryption